In [1]:
# Importing libraries and modules

import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
from sklearn.cluster import KMeans
import cartopy.crs as ccrs
import matplotlib.patches as mpatches
from shapely.affinity import translate

In [2]:
# Reading the file with metrices
df_parking = gpd.read_file("F:/Berlin-DOP_2020/Processed images_new/Final/Pred_distance_calculated.geojson", crs='EPSG:25833')

In [3]:
# Renaming the column heading
df_parking = df_parking.rename(columns={'True or False': 'True_or_False'})

In [4]:
# Adding a buffer of 1 meters to parking polygons
df_parking.geometry = df_parking.geometry.buffer(1)

In [5]:
df_parking

,id_x,label,parking_area_m2,nearest_geometry_dis,geometry
0,1,PK-space-occupied,10.5250,9.398797,"POLYGON ((391033.600 5824043.550, 391032.800 5..."
1,2,PK-space-occupied,12.2575,26.399800,"POLYGON ((391037.110 5824009.979, 391036.217 5..."
2,4,PK-space-occupied,20.5600,1.126644,"POLYGON ((391029.366 5824041.150, 391029.398 5..."
3,3,PK-space-occupied,11.0450,10.902318,"POLYGON ((391018.550 5824051.000, 391020.850 5..."
4,5,PK-space-occupied,14.6600,1.126644,"POLYGON ((391031.536 5824034.864, 391030.336 5..."
...,...,...,...,...,...
8971,8980,PK-space-occupied,2.9750,21.742114,"POLYGON ((392978.800 5824893.250, 392978.400 5..."
8972,8981,PK-space-occupied,7.9675,2.433225,"POLYGON ((392994.200 5824926.750, 392993.600 5..."
8973,8982,PK-space-occupied,18.2300,3.853211,"POLYGON ((392993.700 5824931.450, 392993.300 5..."
8974,8983,PK-space-occupied,12.6250,21.742114,"POLYGON ((392958.625 5824906.988, 392957.925 5..."


In [6]:
# Creating a new fataframe that has union of overlapping geometries
geoms = df_parking.geometry.unary_union
df = gpd.GeoDataFrame(geometry=[geoms])
df = df.explode().reset_index(drop=True)
df.set_geometry = df.geometry
df.crs = {'init' :'epsg:25833'}

C:\ProgramData\Anaconda3\envs\geopandas\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [7]:
#df_parking = df_parking.drop(["index_right"], axis=1)

In [8]:
df.crs

<Projected CRS: EPSG:25833>
Name: ETRS89 / UTM zone 33N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Europe - 12°E to 18°E and ETRS89 by country
- bounds: (12.0, 46.4, 18.01, 84.01)
Coordinate Operation:
- name: UTM zone 33N
- method: Transverse Mercator
Datum: European Terrestrial Reference System 1989
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [9]:
df_parking = df_parking.rename(columns={'id_x': 'id'})

In [10]:
# Taking the spatial join of the dataframes
merged_df = gpd.sjoin(df,df_parking, how="inner")

merged_df["parking_area_m2"] = merged_df['geometry'].area
# Saving the file
merged_df.to_file("F:/Berlin-DOP_2020/Processed images_new/Final.gpkg", Layer= "Predict_dissolved",crs='EPSG:25833',driver="GPKG")